# eda001_1stEDA

In [4]:
import os
import sys
import subprocess
from dotenv import load_dotenv
load_dotenv
sys.path.append(os.getenv('UTILS_PATH'))
from tqdm import tqdm
import multiprocessing

import pandas as pd

In [5]:
INPUT_DIR = os.getenv('INPUT_DIR')
OUTPUT_DIR = os.getenv('OUTPUT_DIR')
PREP_DIR = os.getenv("PREP_DIR")

In [7]:
def ground_truth(events):
    prev_labels = {"clicks": None, "carts": set(), "orders": set()}

    for event in reversed(events):
        event["labels"] = {}

        for label in ['clicks', 'carts', 'orders']:
            if prev_labels[label]:
                if label != 'clicks':
                    event["labels"][label] = prev_labels[label].copy()
                else:
                    event["labels"][label] = prev_labels[label]

        if event["type"] == "clicks":
            prev_labels['clicks'] = event["aid"]
        if event["type"] == "carts":
            prev_labels['carts'].add(event["aid"])
        elif event["type"] == "orders":
            prev_labels['orders'].add(event["aid"])

    return events[:-1]

In [8]:
# データ読み込み
train_sessions = pd.read_pickle(PREP_DIR + "train_sessions.pkl")
test_sessions = pd.read_pickle(PREP_DIR + "test_sessions.pkl")
sub = pd.read_csv(INPUT_DIR + "sample_submission.csv", usecols=["session_type"])

In [27]:
print("train_start : ", pd.to_datetime(train_sessions["ts"], unit="ms").min())
print("train_end : ", pd.to_datetime(train_sessions["ts"], unit="ms").max())
print("test_start : ", pd.to_datetime(test_sessions["ts"], unit="ms").min())
print("test_end : ", pd.to_datetime(test_sessions["ts"], unit="ms").max())

train_start :  2022-07-31 22:00:00.025000
train_end :  2022-08-28 21:59:59.984000
test_start :  2022-08-28 22:00:00.278000
test_end :  2022-09-04 21:59:51.563000


In [10]:
chunks = pd.read_json(INPUT_DIR + "train.jsonl", lines=True, chunksize=1)

In [12]:
a = next(chunks)

In [19]:
events = a["events"][0]

In [20]:
ground_truth(events)

[{'aid': 1517085,
  'ts': 1659304800025,
  'type': 'clicks',
  'labels': {'clicks': 1563459,
   'carts': {275288,
    280978,
    315914,
    442293,
    461689,
    543308,
    789245,
    974651,
    1199474,
    1521766,
    1549618,
    1649869,
    1760145},
   'orders': {305831, 461689, 543308, 1199474}}},
 {'aid': 1563459,
  'ts': 1659304904511,
  'type': 'clicks',
  'labels': {'clicks': 1309446,
   'carts': {275288,
    280978,
    315914,
    442293,
    461689,
    543308,
    789245,
    974651,
    1199474,
    1521766,
    1549618,
    1649869,
    1760145},
   'orders': {305831, 461689, 543308, 1199474}}},
 {'aid': 1309446,
  'ts': 1659367439426,
  'type': 'clicks',
  'labels': {'clicks': 16246,
   'carts': {275288,
    280978,
    315914,
    442293,
    461689,
    543308,
    789245,
    974651,
    1199474,
    1521766,
    1549618,
    1649869,
    1760145},
   'orders': {305831, 461689, 543308, 1199474}}},
 {'aid': 16246,
  'ts': 1659367719997,
  'type': 'clicks',
 

In [7]:
test_sessions = jsonl2df(INPUT_DIR + "test.jsonl")

100%|██████████| 168/168 [40:21<00:00, 14.41s/it]  


In [10]:
pd.read_csv(INPUT_DIR + "sample_submission.csv")

,session_type,labels
0,12899779_clicks,129004 126836 118524
1,12899779_carts,129004 126836 118524
2,12899779_orders,129004 126836 118524
3,12899780_clicks,129004 126836 118524
4,12899780_carts,129004 126836 118524
...,...,...
5015404,14571580_carts,129004 126836 118524
5015405,14571580_orders,129004 126836 118524
5015406,14571581_clicks,129004 126836 118524
5015407,14571581_carts,129004 126836 118524


In [14]:
test_sessions = test_sessions.sort_values(["session", "ts"])
test_sessions = test_sessions.reset_index(drop=True)

In [16]:
test_sessions.to_pickle(PREP_DIR + "test_sessions.pkl")